In [ ]:
%matplotlib inline
import utils; reload(utils)
from utils import plots

# Create Validation set

* 目前採用randomized的產生一組新的train/valid set
* 需去了解大部分介紹CNN的課程中，前期的Cross-validation似乎沒這麼重要，或許是因為做一次的training就已經能找到不太會overfitting的parameters，且一次training所耗費的資源極高

In [ ]:
DATAPATH = 'data/redux'

In [ ]:
%mkdir -p $DATAPATH/sample/train
%mkdir -p $DATAPATH/sample/valid
%mkdir -p $DATAPATH/sample/test
%mkdir -p $DATAPATH/valid

In [ ]:
import os
from glob import glob
from shutil import copy
import numpy as np

根據各個class在原始training set的比例來分配validation set

In [ ]:
dglob = glob(os.path.join(DATAPATH, 'train') + "/dog.*.jpg")
cglob = glob(os.path.join(DATAPATH, 'train') + "/cat.*.jpg")

r_dglob = np.random.permutation(dglob)
r_cglob = np.random.permutation(cglob)

In [ ]:
for i in range(len(r_dglob) / 4): os.rename(r_dglob[i], r_dglob[i].replace('train', 'valid'))
for i in range(len(r_cglob) / 4): os.rename(r_cglob[i], r_cglob[i].replace('train', 'valid'))

In [ ]:
trg = glob(os.path.join(DATAPATH, 'train') + "/*.jpg")
r_trg = np.random.permutation(trg)

vag = glob(os.path.join(DATAPATH, 'valid') + "/*.jpg")
r_vag = np.random.permutation(vag)

teg = glob(os.path.join(DATAPATH, 'test') + "/*.jpg")
r_teg = np.random.permutation(teg)

In [ ]:
for i in range(20): copy(r_trg[i], os.path.join(DATAPATH, 'sample/train'))
for i in range(20): copy(r_vag[i], os.path.join(DATAPATH, 'sample/valid'))
for i in range(10): copy(r_teg[i], os.path.join(DATAPATH, 'sample/test'))

## 將不同類別放進不同資料夾中

In [ ]:
%mkdir -p $DATAPATH/train/dogs
%mkdir -p $DATAPATH/train/cats
%mkdir -p $DATAPATH/valid/dogs
%mkdir -p $DATAPATH/valid/cats
%mkdir -p $DATAPATH/test/unknown

%mkdir -p $DATAPATH/sample/train/dogs
%mkdir -p $DATAPATH/sample/train/cats
%mkdir -p $DATAPATH/sample/valid/dogs
%mkdir -p $DATAPATH/sample/valid/cats
%mkdir -p $DATAPATH/sample/test/unknown

# the code below cannot be executed because the command "mv" has argument limitation in macOS
# use "find" instead
#%mv $DATAPATH/train/dog.*.jpg $DATAPATH/train/dogs
#%mv $DATAPATH/train/cat.*.jpg $DATAPATH/train/cats
#%mv $DATAPATH/valid/dog.*.jpg $DATAPATH/valid/dogs
#%mv $DATAPATH/valid/cat.*.jpg $DATAPATH/valid/cats
#%mv $DATAPATH/test/*.jpg $DATAPATH/test/unknown

!find train -name 'dog.*.jpg' | xargs -I {} mv {} train/dogs/
!find train -name 'cat.*.jpg' | xargs -I {} mv {} train/cats/
!find valid -name 'dog.*.jpg' | xargs -I {} mv {} valid/dogs/
!find valid -name 'cat.*.jpg' | xargs -I {} mv {} valid/cats/
!find test -name '*.jpg' | xargs -I {} mv {} test/unknown/


%mv $DATAPATH/sample/train/dog.*.jpg $DATAPATH/sample/train/dogs
%mv $DATAPATH/sample/train/cat.*.jpg $DATAPATH/sample/train/cats
%mv $DATAPATH/sample/valid/dog.*.jpg $DATAPATH/sample/valid/dogs
%mv $DATAPATH/sample/valid/cat.*.jpg $DATAPATH/sample/valid/cats
%mv $DATAPATH/sample/test/*.jpg $DATAPATH/sample/test/unknown


這裡可以發現，如果使用Keras的batch來讀image，會花相當多的時間在搬動以及分類照片到不同的資料夾中，如果還要使用Pyhton的library做partition，相當的麻煩，或許可以考慮寫一隻script來完成以上的工作。

In [ ]:
data_path = 'data/redux/'
result_path = 'result/'

# 透過Fune-tune的方式來訓練CNN

In [ ]:
from vgg16 import Vgg16

### Initial Vgg16

In [ ]:
vgg = Vgg16()
batch_size=64

In [ ]:
batches = vgg.get_batches(data_path + "train", batch_size = batch_size)
val_batches = vgg.get_batches(data_path + "valid", batch_size = batch_size)
vgg.finetune(batches)

必須在fine-tune過後才可以加上optimizer???
```AttributeError: 'Sequential' object has no attribute 'optimizer'```

### Model parameter

In [ ]:
no_of_epochs = 3
vgg.model.optimizer.lr = 0.01

### Fit

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)
weights_filename = 'ft%d.h5' % epoch
vgg.model.save_weights(result_path + weights_filename)

# Prediction & Visualization

In [ ]:
vgg.model.load_weights('ft0.h5')
test_batches, pred = vgg.test(data_path + 'test', batch_size=batch_size)

In [ ]:
import re
import pandas as pd
pred = pd.DataFrame({'label': pred[:,1], 'file' : test_batches.filenames})
pred['id'] = pred.apply(lambda x: re.sub('[^0-9]', '', x['file']), axis = 1)
pred['id'] = pred['id'].astype('int32')
submission_df = pred.sort_values(by=['id'])[['id', 'label']]
submission_df.to_csv('sample_submission.csv', index=False)